# Analysis of the AlphaFold predictions of four-domain MetH
## 3. Align the structures in pymol so they have the same relative coordinates
### You need to install pymol module (see Readme.md).

In [1]:
import pymol
import pandas as pd
import pickle

Cannot find license file.
 The license files (or license server system network addresses) attempted are 
listed below.  Use LM_LICENSE_FILE to use a different license file,
 or contact your software provider for a license file.
Feature:       PYMOL_MAIN
Filename:      /Library/Application Support/Schrodinger/licenses
License path:  /Library/Application Support/Schrodinger/licenses:
FlexNet Licensing error:-1,359.  System Error: 2 "No such file or directory"


In [2]:
# load the variables
B12_f = open('B12_ps_string', 'rb')
B12_ps_string = pickle.load(B12_f) # serialize the list
B12_f.close()

SAM_f = open('SAM_ps_string', 'rb')
SAM_ps_string = pickle.load(SAM_f) # serialize the list
SAM_f.close()

Zinc_f = open('Zinc_ps_string', 'rb')
Zinc_ps_string = pickle.load(Zinc_f) # serialize the list
Zinc_f.close()

MetH_4d_info = pd.read_pickle("MetH_4d_info.pkl")

In [3]:
# find the center of mass  of every domain and the coordinates of the binding sites when aligned to the E.coli predicted structure (P13009)

DomainP_clean = MetH_4d_info
pymol.cmd.reinitialize()
com0 = [];
Binding_sites_c = [];
rmsd_align = [];
id_ref = 'P13009'
Directory_pdb = './MetH_Alphafold_database_4d'
for i in range(0,len(DomainP_clean)):
    
    # Delete everything
    pymol.cmd.delete('all')
    
    # load domain and binding sites position information
    id1 = DomainP_clean.Uniprot_ID.iloc[i]
    Hcy_1 = DomainP_clean.Hcy_1.iloc[i]
    Hcy_2 = DomainP_clean.Hcy_2.iloc[i]
    MTHF_1 = DomainP_clean.MTHF_1.iloc[i]
    MTHF_2 = DomainP_clean.MTHF_2.iloc[i]
    cap_1 = DomainP_clean.cap_1.iloc[i]
    cap_2 = DomainP_clean.cap_2.iloc[i]
    B12_1 = DomainP_clean.B12_1.iloc[i]
    B12_2 = DomainP_clean.B12_2.iloc[i]
    AdoMet_1 = DomainP_clean.AdoMet_1.iloc[i]
    AdoMet_2 = DomainP_clean.AdoMet_2.iloc[i]
    Zinc_binding = Zinc_ps_string[i]
    B12_binding = B12_ps_string[i]
    SAM_binding = SAM_ps_string[i]
    
    # load the pdb file
    pymol.cmd.load(f'{Directory_pdb}/{id1}.pdb')
    
    # align the structure with the reference structure
    if not id1==id_ref:
        pymol.cmd.load(f'{Directory_pdb}/{id_ref}.pdb')
        data = pymol.cmd.align(f'{id1} and resi {Hcy_1}-{MTHF_2}',f'{id_ref}')
        rmsd_align.append(data[0])
        pymol.cmd.delete(f'{id_ref}')
    # Select every domain
    pymol.cmd.select(f'Hcy{i}', f'(resi {Hcy_1}-{Hcy_2})')
    pymol.cmd.select(f'MTHF{i}', f'(resi {MTHF_1}-{MTHF_2})')
    pymol.cmd.select(f'cap{i}', f'(resi {cap_1}-{cap_2})')
    pymol.cmd.select(f'B12{i}', f'(resi {B12_1}-{B12_2})')
    pymol.cmd.select(f'AdoMet{i}', f'(resi {AdoMet_1}-{AdoMet_2})')
    
    # Select every binding site
    pymol.cmd.select(f'Zinc_b{i}', f'(resi {Zinc_binding})')
    pymol.cmd.select(f'B12_b{i}', f'(resi {B12_binding})')
    pymol.cmd.select(f'SAM_b{i}', f'(resi {SAM_binding})')

    
    # Get the coordinates of the binding sites
    Zinc_c_s = pymol.cmd.get_coords(f'Zinc_b{i}',state=1)
    B12_c_s = pymol.cmd.get_coords(f'B12_b{i}',state=1)
    SAM_c_s = pymol.cmd.get_coords(f'SAM_b{i}',state=1)
    
    # Get the center of mass of every domain
    com1 = pymol.cmd.centerofmass(f'Hcy{i}',state=1)
    com2 = pymol.cmd.centerofmass(f'MTHF{i}',state=1)
    com3 = pymol.cmd.centerofmass(f'cap{i}',state=1)
    com4 = pymol.cmd.centerofmass(f'B12{i}',state=1)
    com5 = pymol.cmd.centerofmass(f'AdoMet{i}',state=1)
    
    print(com1,com2,com3,com4,com5)
    print(pymol.cmd.get_names('all'))
    
    # append the value to save
    com0.append([com1,com2,com3,com4,com5])
    Binding_sites_c.append([Zinc_c_s,B12_c_s,SAM_c_s])


[19.43636903671909, -4.006812730559882, 1.279412765152721] [-8.416276825452233, -17.31515774453417, 26.56153200066399] [-32.55146594373695, -3.192940873691364, -8.297293872168414] [-10.887341270051579, 14.618014808738339, -17.51516861051781] [-8.676838920304704, 36.080373245483884, 5.743637564109187]
['A0A011PTV8', '_align1', '_align2', 'Hcy0', 'MTHF0', 'cap0', 'B120', 'AdoMet0', 'Zinc_b0', 'B12_b0', 'SAM_b0']
[19.4656088374439, -4.016829938491231, 1.745444606338612] [-8.304797483927011, -17.258408931260096, 26.354419673992396] [-30.039053572130612, 3.4883952757758507, -7.771643342275017] [-11.530151015488245, 12.885415505018493, -29.516808023979426] [17.90444261031018, 5.8132848107335136, -22.505373325707918]
['A0A011QQ52', '_align1', '_align2', 'Hcy1', 'MTHF1', 'cap1', 'B121', 'AdoMet1', 'Zinc_b1', 'B12_b1', 'SAM_b1']
[19.63562452167952, -4.0057480025191285, 1.0293139904607387] [-8.276629933400313, -17.275205486253068, 26.766145811854667] [-32.889761528904785, 2.6071866078332744, -1.

In [4]:
# store the variables to pkl files
com0_f = open('com0', 'wb')
pickle.dump(com0, com0_f) # serialize the list
com0_f.close()

Bc_f = open('Binding_sites_c', 'wb')
pickle.dump(Binding_sites_c, Bc_f) # serialize the list
Bc_f.close()